# fineDEMfoam Solver Analysis

## 1. Fluid Force Calculation
The fluid force acting on a particle is determined using a volume-integrated approach, consistent with the Immersed Boundary Method (IBM). The calculation is performed in `coupling.H` and relies on fields calculated in `calculateIBM.H`.

### A. Particle Representation (IBM)
In `calculateIBM.H`, the presence of particles is mapped onto the fluid mesh using a solid volume fraction field, $\epsilon$ (`epsilon`).
- **Support Radius**: $R_{support} = 1.05 \times R_{particle}$.
- **Kernel Function**: A linear kernel function $w(d)$ is used:
  $$ w = 1.0 - \frac{d}{R_{support}} $$
- **Normalization**: Weights are normalized so the integral of $\epsilon$ equals the particle volume.

### B. Force Components
The total fluid force is the sum of buoyancy and viscous forces, integrated over cells where $\epsilon > 0$.

#### i. Pressure Force (Buoyancy)
Calculated by integrating the negative pressure gradient over the particle's volume.
$$ \mathbf{F}_p = \sum_{cells} \left( -\nabla p \cdot \epsilon_{cell} \cdot V_{cell} \right) $$
In code (`coupling.H`):
```cpp
volVectorField gradP = fvc::grad(p_rgh) + rho*g;
vector fb = -gradP[celli] * vol;
```

#### ii. Viscous Force (Drag)
Calculated by integrating the viscous stress term (approximated by Laplacian of U).
$$ \mathbf{F}_v = \sum_{cells} \left( \mu \nabla^2 \mathbf{U} \cdot \epsilon_{cell} \cdot V_{cell} \right) $$
In code (`coupling.H`):
```cpp
volVectorField lapUField = fvc::laplacian(U);
vector fv = mu * lapU * vol;
```

#### iii. Gravity
$$ \mathbf{F}_g = m_{particle} \mathbf{g} $$

### C. Total Force
$$ \mathbf{F}_{total} = \mathbf{F}_p + \mathbf{F}_v + \mathbf{F}_g $$

## 2. Fluid Equations
The solver solves the multiphase Navier-Stokes equations with an IBM penalty term.

### A. Momentum Equation (`UEqn.H`)
Includes a penalty source term to enforce no-slip at the particle surface.
$$ \frac{\partial (\rho \mathbf{U})}{\partial t} + \nabla \cdot (\rho \phi \mathbf{U}) + \nabla \cdot \tau + S_p \mathbf{U} = S_p \mathbf{U}_{solid} $$
- **Penalty Coefficient**: $S_p = \epsilon \cdot \rho \cdot C_{penalty}$ ($C_{penalty} \approx 10^5$).

### B. Pressure Equation (`pEqn.H`)
Solves for dynamic pressure $p_{rgh}$ using PIMPLE algorithm.
$$ \nabla \cdot \left( \frac{1}{A_U} \nabla p_{rgh} \right) = \nabla \cdot \phi^* $$

### C. Phase Fraction Equation (`alphaEqn.H`)
Solves VOF equation using MULES.
$$ \frac{\partial \alpha_1}{\partial t} + \nabla \cdot (\mathbf{U} \alpha_1) + \nabla \cdot (\mathbf{U}_r \alpha_1 (1-\alpha_1)) = 0 $$

## 3. What is $\phi^*$?
In the pressure equation, $\phi^*$ (denoted as `phiHbyA` in OpenFOAM code) represents the **predicted mass flux** at the cell faces, constructed from the momentum equation **excluding the pressure gradient**.

Mathematically:
$$ \phi^* = \left( \frac{\mathbf{H}(\mathbf{U})}{A_U} \right)_f \cdot \mathbf{S}_f + \phi_{gravity} + \phi_{surface\_tension} $$

Where:
- $\mathbf{H}(\mathbf{U})$: Transport part of momentum equation (convection, diffusion, sources) without pressure gradient.
- $A_U$: Diagonal coefficient of momentum matrix.
- $\phi_{gravity}$: Flux due to gravity.

The pressure equation corrects this flux to enforce continuity:
$$ \nabla \cdot \phi_{corrected} = 0 $$
$$ \phi_{corrected} = \phi^* - \left( \frac{1}{A_U} \nabla p_{rgh} \right)_f \cdot \mathbf{S}_f $$

## 4. Turbulence Model Equations
The solver uses the `incompressibleInterPhaseTransportModel` library, allowing runtime selection of turbulence models (RAS or LES). A standard choice is the **k-Epsilon ($k-\epsilon$)** model.

### Standard $k-\epsilon$ Equations
**1. Turbulent Kinetic Energy ($k$):**
$$ \frac{\partial (\rho k)}{\partial t} + \nabla \cdot (\rho \mathbf{U} k) = \nabla \cdot \left[ \left( \mu + \frac{\mu_t}{\sigma_k} \right) \nabla k \right] + P_k - \rho \epsilon $$

**2. Turbulent Dissipation Rate ($\epsilon$):**
$$ \frac{\partial (\rho \epsilon)}{\partial t} + \nabla \cdot (\rho \mathbf{U} \epsilon) = \nabla \cdot \left[ \left( \mu + \frac{\mu_t}{\sigma_\epsilon} \right) \nabla \epsilon \right] + C_{1\epsilon} \frac{\epsilon}{k} P_k - C_{2\epsilon} \rho \frac{\epsilon^2}{k} $$

**3. Turbulent Viscosity:**
$$ \mu_t = \rho C_\mu \frac{k^2}{\epsilon} $$

Where $P_k$ is the production of turbulent kinetic energy due to mean velocity gradients.

## 5. Immersed Boundary Method (IBM) Equations
After obtaining the particle geometries, the IBM implementation proceeds in three main steps: Mapping, Velocity Reconstruction, and Momentum Forcing.

### A. Mapping (Solid Volume Fraction)
The presence of particles is mapped onto the Eulerian fluid mesh using a **Solid Volume Fraction field ($\epsilon$)**. A linear kernel function distributes the particle's volume to surrounding fluid cells.

For a particle $p$ with center $\mathbf{x}_p$ and support radius $R_{support}$ (where $R_{support} \approx 1.05 R_p$), the weight $w_{i}$ for a mesh cell $i$ at position $\mathbf{x}_i$ is:

$$ w_{i} = \begin{cases} 1 - \frac{|\mathbf{x}_i - \mathbf{x}_p|}{R_{support}} & \text{if } |\mathbf{x}_i - \mathbf{x}_p| < R_{support} \\ 0 & \text{otherwise} \end{cases} $$

The solid volume fraction $\epsilon_i$ in cell $i$ is then computed by normalizing these weights to conserve the particle's volume $V_p$:

$$ \epsilon_i = \sum_{p} \left( w_{i,p} \cdot \frac{V_p}{\sum_{j} w_{j,p} V_j} \right) $$

### B. Solid Velocity Reconstruction
The velocity of the solid phase ($\mathbf{U}_{solid}$) is constructed at every fluid cell $i$ that lies within the support radius of a particle. It combines the particle's translational ($\mathbf{U}_p$) and angular ($\boldsymbol{\omega}_p$) velocities:

$$ \mathbf{U}_{solid, i} = \mathbf{U}_p + \boldsymbol{\omega}_p \times (\mathbf{x}_i - \mathbf{x}_p) $$

### C. Momentum Forcing (Penalty Method)
The IBM interaction is enforced by adding a **penalty source term** ($\mathbf{f}_{IBM}$) to the fluid momentum equation. This forces the fluid velocity $\mathbf{U}$ to match the solid velocity $\mathbf{U}_{solid}$ in regions occupied by the particle ($\epsilon > 0$).

The modified momentum equation is:

$$ \frac{\partial (\rho \mathbf{U})}{\partial t} + \nabla \cdot (\rho \mathbf{U}\mathbf{U}) = -\nabla p + \nabla \cdot \boldsymbol{\tau} + \mathbf{f}_{IBM} $$

The forcing term is modeled as:

$$ \mathbf{f}_{IBM} = S_p (\mathbf{U}_{solid} - \mathbf{U}) $$

Where $S_p$ is the **penalty coefficient**, defined as:

$$ S_p = \epsilon \cdot \rho \cdot C_{penalty} $$

- $\epsilon$: Solid volume fraction (0 to 1).
- $\rho$: Fluid density.
- $C_{penalty}$: A large constant (set to $10^5$ in `UEqn.H`) to ensure the constraint is strictly enforced.